In [1]:
conda install -c conda-forge scikit-surprise

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done



In [2]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd

#pd.options.display.max_columns = None
#pd.options.display.max_rows = None
#pd.options.display.max_colwidth = None
#pd.options.display.precision = 20

import heapq
from collections import defaultdict

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

#pd.options.display.max_rows = 10

# RESET AIR

In [4]:
a, i, r = (pd.read_csv("student_movie_attributes.csv"),
           pd.read_csv("student_movie_intent.csv"),
           pd.read_csv("student_movie_ratings.csv"))

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
a, i, r = (a.drop(['etl_run_guid', 'intent_score', 'survey_date', 'content_type', 'movie_name',
                   'household_size', 'gender_name', 'age_group_bracket', 'ethnicity_name',
                   'education_level', 'income_bracket'], axis=1),
           i.drop(['etl_run_guid'], axis=1),
           r.drop(['etl_run_guid', 'networks', 'content_type'], axis=1))

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
a, i, r = (a.sort_values(by=['respondent_id', 'movie_id']),
           i.sort_values(by=['respondent_id', 'movie_id']),
           r.sort_values(by=['respondent_id', 'movie_id']))

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
ir = r.merge(i, how='outer', on=['respondent_id', 'movie_id'])

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
air = ir.merge(a, how='outer', on=['respondent_id', 'movie_id'])
air = air.dropna()

print('(Rows, Columns)', air.shape)
print('Any null values?', air.isnull().values.any())
air

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(Rows, Columns) (249284, 74)
Any null values? False


,survey_date,respondent_id,movie_id,movie_name,gender_name,age_group_bracket,ethnicity_name,education_level,income_bracket,household_size,...,exciting,calming,slow,unoriginal,uplifting,sexy,deep,suspenseful,thrilling,scary
0,1/18/2020,743006,11549,Ad Astra,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1/18/2020,743006,11679,Beetlejuice,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1/18/2020,743006,11692,Deliverance,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,1/18/2020,743006,11708,Gremlins,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1/18/2020,743006,11748,Risky Business,Male,55 to 64,Hispanic or Latino,Some college,"$75,000 - $99,999",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256299,6/11/2020,956985,9111,Abducted: The Carlina White Story,Female,65 and Over,African-American or Black,High school diploma,"$25,000 - $34,999",1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
256301,6/12/2020,957051,9111,Abducted: The Carlina White Story,Female,65 and Over,White or Caucasian,High school diploma,"$15,000 - $24,999",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
256302,6/11/2020,957162,9111,Abducted: The Carlina White Story,Male,18 to 24,African-American or Black,College degree,"$50,000 - $74,999",3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
256303,6/12/2020,957166,9111,Abducted: The Carlina White Story,Male,18 to 24,White or Caucasian,High school diploma,"$25,000 - $34,999",1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
air.movie_name.nunique()

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3213

# STOP RESET

# COLLABORATIVE FILTERING

In [10]:
cf = air[['respondent_id', 'movie_name', 'rating']]
cf

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,respondent_id,movie_name,rating
0,743006,Ad Astra,4
1,743006,Beetlejuice,5
2,743006,Deliverance,5
3,743006,Gremlins,5
4,743006,Risky Business,3
...,...,...,...
256299,956985,Abducted: The Carlina White Story,4
256301,957051,Abducted: The Carlina White Story,2
256302,957162,Abducted: The Carlina White Story,4
256303,957166,Abducted: The Carlina White Story,3


In [11]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(cf[['respondent_id', 'movie_name', 'rating']], reader)
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': False}  # compute cosine similarities between items
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Computing the cosine similarity matrix...


C:\Users\larry\anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sim = construction_func[name](*args)


Done computing similarity matrix.


In [12]:
cf.movie_name.nunique()

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3213

In [13]:
movies = cf.movie_name.unique()
resp = cf.respondent_id.unique()

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
#for movie in movies: 
#    print(movie, algo.predict(783008, movie).est)

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# RECOMMENDATION FUNCTION

In [15]:
#TOP 10 RECOMMENDATIONS

def recommendation_10(ID):
    pd.options.display.precision = 2
    rcmd = pd.DataFrame(columns=['Movie_Name', 'Rating'])

    for movie in movies:
        rcmd = rcmd.append({'Movie_Name': movie, 'Rating': algo.predict(ID, movie).est}, ignore_index=True)
     
        rcmd = rcmd.sort_values(by=['Rating'], ascending=False).head(10)
    title = 'Respondent ' + str(ID) + "'s Top 10 Recommended Movies", rcmd
    return title

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Input ID Range from 743006 to 791017

recommendation_10(743006)

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


("Respondent 743006's Top 10 Recommended Movies",
                            Movie_Name  Rating
 0        Sharknado 5: Global Swarming     5.0
 1                   National Treasure     5.0
 2  National Treasure: Book of Secrets     5.0
 3                  Return of the Jedi     5.0
 4              A Good Day to Die Hard     5.0
 5                     The Lone Ranger     5.0
 6              Boo! A Madea Halloween     5.0
 7                               Sully     5.0
 8                          The Intern     5.0
 9                        Wonder Woman     5.0)

In [17]:
#RECOMMENDS MOVIES WITH RATINGS = 5

def recommendation_5(ID):
    pd.options.display.max_rows = None
    pd.options.display.precision = 2
    rcmd = pd.DataFrame(columns=['Movie_Name', 'Rating'])

    for movie in movies:
        rcmd = rcmd.append({'Movie_Name': movie, 'Rating': algo.predict(ID, movie).est}, ignore_index=True)
        rcmd = rcmd.sort_values(by=['Rating'], ascending=False)
        rcmd = rcmd.loc[rcmd['Rating'] == 5]
    title = 'Respondent ' + str(ID) + ':' + ' ' + str(len(rcmd.index)) +' Movies Likely to Rate 5',rcmd
    return title

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Input ID Range from 743006 to 791017

#recommendation_5(743006)

C:\Users\larry\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
